In [3]:
from itertools import chain

In [60]:
import sys
import warnings
import time

from tqdm import tqdm
from bs4 import BeautifulSoup

import pyderman

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from pprint import pprint

import json

In [5]:
def open_chrome(headless=True):
    path = pyderman.install(browser=pyderman.chrome)

    options = Options()
    options.add_argument("--enable-javascript")
    options.headless = headless

    driver = webdriver.Chrome(path, options=options)

    # Sanity checks.
    driver.get("http://www.python.org")
    assert "Python" in driver.title
    return driver



In [78]:
driver = open_chrome(False)

chromedriver is already installed.


In [79]:
gscholar_url = "https://scholar.google.com/citations?user=qre6vl0AAAAJ&hl=en"
driver.get(gscholar_url)

In [80]:
while True:
    e = driver.find_element_by_xpath('//button[@id="gsc_bpf_more"]')
    e.click()
    if e.get_attribute('disabled') == 'true':
        break

In [102]:
publications = []

for e in driver.find_elements_by_xpath('//tr[@class="gsc_a_tr"]'):
    WebDriverWait(e, 20).until(EC.element_to_be_clickable((By.XPATH, 
        '//a[(@class="gsc_a_at") and contains(text(), "{}")]'.format(e.text.split('\n')[0])))).click()
    _json = {}
    ##time.sleep(0.7)
    print(e.text)
    bsoup = BeautifulSoup(driver.page_source.encode('utf-8'))
    cite = bsoup.find('div', attrs={'id': 'gs_md_cita-l'})
    keys = cite.find_all('div', attrs={'class':'gsc_vcd_field'})
    values = cite.find_all('div', attrs={'class':'gsc_vcd_value'})
    for k,v in zip(keys, values):
        if k.text == 'Scholar articles':
            continue
        if k.text == 'Total citations':
            _json[k.text] = int(v.find('a').text[9:])
        else:
            _json[k.text] = v.get_text('\n')
    ##print(_json)
    time.sleep(0.7)
    driver.find_element_by_xpath('//a[@id="gs_md_cita-d-x"]').click()

    if _json:
        json_str = json.dumps(_json)
        if json_str not in publications:
            publications.append(json_str)
    ##print('#######')
publications = [json.loads(c) for c in publications]


A report on the DSL shared task 2014
M Zampieri, L Tan, N Ljubešić, J Tiedemann
Proceedings of the first workshop on applying NLP tools to similar languages …
80 2014
{'Authors': 'Marcos Zampieri, Liling Tan, Nikola Ljubešić, Jörg Tiedemann', 'Publication date': '2014/8', 'Conference': 'Proceedings of the first workshop on applying NLP tools to similar languages, varieties and dialects', 'Pages': '58-67', 'Description': 'This paper summarizes the methods, results and findings of the Discriminating between Similar Languages (DSL) shared task 2014. The shared task provided data from 13 different languages and varieties divided into 6 groups. Participants were required to train their systems to discriminate between languages on a training and development set containing 20,000 sentences from each language (closed submission) and/or any other dataset (open submission). One month later, a test set containing 1,000 unidentified instances per language was released for evaluation. The DSL share

#######
Grammatical error detection with limited training data: The case of chinese
M Zampieri, L Tan
Proceedings of ICCE
10 2014
{'Authors': 'Marcos Zampieri, Liling Tan', 'Publication date': '2014/11/30', 'Journal': 'Proceedings of ICCE', 'Description': 'In this paper, we describe the UDS submission to the shared task on Grammatical Error Diagnosis for Learning Chinese as a Foreign Language. We designed four different experiments (runs) to approach this task. All of them are variations of a frequency-based approach using a journalistic corpus as standard corpus and comparing n-gram frequency lists to both the training and the test corpus provided by the shared task organizers. The assumption behind this approach is that comparing a standard reference corpus to a non-standard study corpus using frequency-based methods levels out non-standard features present in the study corpus. These features are very likely to be, in the case of this corpus, grammatical errors. Our system obtained 6

#######
Passive and pervasive use of a bilingual dictionary in statistical machine translation
L Tan, J van Genabith, F Bond
6 2015
{'Authors': 'Liling Tan, Josef van Genabith, Francis Bond', 'Publication date': '2015/7/1', 'Description': 'There are two primary approaches to the use bilingual dictionary in statistical machine translation: (i) the passive approach of appending the parallel training data with a bilingual dictionary and (ii) the pervasive approach of enforcing translation as per the dictionary entries when decoding. Previous studies have shown that both approaches provide external lexical knowledge to statistical machine translation thus improving translation quality. We empirically investigate the effects of both approaches on the same dataset and provide further insights on how lexical information can be reinforced in statistical machine translation.', 'Total citations': 6}
#######
An Awkward Disparity between BLEU/RIBES and Human Judgment in Machine Translation
L Tan, 

#######
Saarsheff at semeval-2016 task 1: semantic textual similarity with machine translation evaluation metrics and (extreme) boosted tree ensembles
L Tan, C Scarton, L Specia, J van Genabith
Proceedings of the 10th International Workshop on Semantic Evaluation …
3 2016
{'Authors': 'Liling Tan, Carolina Scarton, Lucia Specia, Josef van Genabith', 'Publication date': '2016/6', 'Conference': 'Proceedings of the 10th International Workshop on Semantic Evaluation (SemEval-2016)', 'Pages': '628-633', 'Description': 'This paper describes the SAARSHEFF systems that participated in the English Semantic Textual Similarity (STS) task in SemEval-2016. We extend the work on using machine translation (MT) metrics in the STS task by automatically annotating the STS datasets with a variety of MT scores for each pair of text snippets in the STS datasets. We trained our systems using boosted tree ensembles and achieved competitive results that outperforms he median Pearson correlation scores from all

#######
Sarah’s Participation in WAT 2019
RH Susanto, O Htun, L Tan
Proceedings of the 6th Workshop on Asian Translation, 152-158
2019
{'Authors': 'Raymond Hendy Susanto, Ohnmar Htun, Liling Tan', 'Publication date': '2019/11', 'Conference': 'Proceedings of the 6th Workshop on Asian Translation', 'Pages': '152-158', 'Description': 'This paper describes our MT systems’ participation in the of WAT 2019. We participated in the (i) Patent,(ii) Timely Disclosure,(iii) Newswire and (iv) Mixed-domain tasks. Our main focus is to explore how similar Transformer models perform on various tasks. We observed that for tasks with smaller datasets, our best model setup are shallower models with lesser number of attention heads. We investigated practical issues in NMT that often appear in production settings, such as coping with multilinguality and simplifying pre-and post-processing pipeline in deployment.'}
#######
Proceedings of the Third Workshop on NLP for Similar Languages, Varieties and Dialect

In [ ]:
# Save the publications in json file.
output_json_filename = 'liling-tan.semscholar.json'
with open(output_json_filename, 'w') as fout:
    json.dump(publications, fout)

In [103]:
gs_collaborators = set(chain(*[c['Authors'].split(', ') for c in publications]))

In [105]:
gs_pubyears = set([c['Publication date'].split('/')[0] for c in publications])

In [106]:
gs_pubyears

{'2011', '2013', '2014', '2015', '2016', '2018', '2019'}